In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
auctions = pd.read_csv("data/auctions.csv.gzip", parse_dates=['date'])
auctions.columns = ['date', 'ref_hash', 'ref_type_id', 'source_id']

In [4]:
clicks = pd.read_csv("data/clicks.csv.gzip")

D:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
#installs = pd.read_csv("data/installs.csv.gzip")

D:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
#events = pd.read_csv("data/events.csv.gzip")

D:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
target = pd.read_csv("data/target_final_competencia_revamped.csv")

In [6]:
auctions.sort_values('date', ascending=False)

,date,ref_hash,ref_type_id,source_id
45963986,2019-04-26 23:59:59.969518,4337086052205446508,1,0
44059221,2019-04-26 23:59:59.962455,2604334459441014255,1,3
18431827,2019-04-26 23:59:59.953129,7696313774830088806,1,0
28210820,2019-04-26 23:59:59.948415,3963832332737333138,1,7
18431826,2019-04-26 23:59:59.898628,652599192004627901,1,0
3801015,2019-04-26 23:59:59.855439,9123029882256516867,7,1
18430235,2019-04-26 23:59:59.825401,1430878265242608404,1,1
4301357,2019-04-26 23:59:59.810858,41673522045424767,1,0
12712359,2019-04-26 23:59:59.799169,988633542015871508,1,3
13743304,2019-04-26 23:59:59.784969,5443807710972817324,1,3


In [7]:
auctions_first = auctions[auctions['date'] < '2019-04-21 00:00:00.000000']
auctions_second = auctions[(auctions['date'] >= '2019-04-21 00:00:00.000000') & (auctions['date'] < '2019-04-24 00:00:00.000000')]
auctions_first_label = auctions_second.sort_values('date').groupby('ref_hash').first().reset_index()
auctions_first_label['time_appearence'] = (auctions_first_label['date'] - pd.to_datetime('2019-04-21 00:00:00.000000')).dt.seconds
auctions_first_label = auctions_first_label.drop(columns=['date', 'ref_type_id', 'source_id'])

In [8]:
clicks_first = clicks[clicks['created'] < '2019-04-21 00:00:00.000000']
clicks_first = pd.merge(clicks_first, clicks_first.groupby('ref_hash').agg('size').reset_index(), on='ref_hash', how='inner')
clicks_first = clicks_first.drop_duplicates('ref_hash')
clicks_first = clicks_first.rename(columns={0:'ammount_clicks'})

In [9]:
auctions = 0

In [10]:
clicks_first = clicks_first[['ref_hash', 'ammount_clicks']]

In [13]:
clicks_first.head()

,ref_hash,ammount_clicks
0,1293710398598742392,7
7,1663930990551616564,2
9,8488038938665586188,6
15,6488361690105189959,3
18,1348993302102753419,1


,date,ref_hash,ref_type_id,source_id
221726,2019-04-20 23:57:27.912838,1109595589636746168,7,0
221727,2019-04-20 23:57:28.381114,5896614299191635403,1,0
221728,2019-04-20 23:57:28.515423,4172466725848941608,1,0
221729,2019-04-20 23:57:28.700884,2616279795187318849,7,0
221730,2019-04-20 23:57:28.868312,8034952072073026056,1,0


In [15]:
auctions_first = pd.merge(auctions_first, auctions_first.groupby('ref_hash').agg('size').reset_index(), on='ref_hash', how='inner')
auctions_first = pd.merge(auctions_first, clicks_first, on='ref_hash', how='left')
auctions_first = auctions_first.drop_duplicates('ref_hash')
auctions_first = pd.merge(auctions_first, auctions_first_label, on='ref_hash', how='inner')
auctions_first['ref_type_id_1'] = auctions_first['ref_type_id'].apply(lambda x: 1 if x==1 else 0)
auctions_first['ref_type_id_7'] = auctions_first['ref_type_id'].apply(lambda x: 1 if x==7 else 0)

In [17]:
auctions_first = auctions_first.drop(columns=['date', 'ref_hash', 'source_id', 'ref_type_id'])

In [22]:
auctions_first.columns = ['appearences_in_auctions', 'ammount_clicks', 'time_appearence', 'ref_type_id_1', 'ref_type_id_7']

In [20]:
auctions_first = auctions_first.fillna(0)

In [23]:
auctions_first.head()

,appearences_in_auctions,ammount_clicks,time_appearence,ref_type_id_1,ref_type_id_7
0,128,0.0,9,0,1
1,46,0.0,562,1,0
2,252,0.0,26746,1,0
3,191,0.0,199,0,1
4,166,0.0,731,1,0


Time_appearence es el label, el valor a predecir por el modelo, los demás son features. Ref_type_id (one hot encoding) indica a que plataforma corresponde, appearences_in_auctions indica la cantidad de apariciones de dicho dispositivo en subastas y ammount_clicks indica la cantidad de clicks.

In [24]:
train, test = train_test_split(auctions_first, test_size=0.2)

In [25]:
train_Y = train['time_appearence']
train_X = train.drop(columns=['time_appearence'])
test_Y = test['time_appearence']
test_X = test.drop(columns=['time_appearence'])

In [26]:
model = tree.DecisionTreeRegressor()

In [114]:
model = model.fit(train_X, train_Y)

In [115]:
mean_squared_error(train_Y, model.predict(train_X))

739602259.7912194

In [116]:
mean_squared_error(test_Y, model.predict(test_X))

776566334.9462426

In [18]:
mean_squared_error(test_Y, model.predict(test_X))

797697299.3978992